# Interconversion between PubChem records.

PUG-REST can be used to retrieve PubChem records related to another PubChem records.  Basically, PUG-REST takes an input list of records in one of the three PubChem databases (Compound, Substance, and BioAssay) and returns a list of the related records in the same or different database.  Here, the meaning of the relationship between the input and output records may be specified using an optional parameter.  This allows one to do various tasks, including (but not limited to):<br>

- Depositor-provided records (i.e., substances) that are standardized to a given compound.
- Mixture compounds that contain a given component compound.
- Stereoisomers/isotopomers of a given compound.
- Compounds that are tested to be active in a given assay.
- Compounds that have similar structures to a given compound.

## 1. Getting depositor-provided records for a given compound

First let's import the requests package necessary to make a web service request.

In [1]:
import requests
import sys
import os

# Get the absolute path to the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

from my_pack import my_fun

The code snippet below retrieves the substance record associated with a given CID (CID 129825914).

In [2]:
prolog    = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"

pr_input  = "compound/cid/129825914"
pr_oper   = "sids"
pr_output = "txt"
url       = prolog + '/' + pr_input + '/' + pr_oper + '/' + pr_output

res = requests.get(url)
print(res.text)

341669951



It is also possible to provide a comma seprated list of CIDs as input identifiers.

In [3]:
pugin   = "compound/cid/129825914,129742624,129783988"
pugoper = "sids"
pugout  = "txt"
url     = prolog + '/' + pugin + '/' + pugoper + '/' + pugout

res = requests.get(url)
print(res.text)

341669951
341492923
341577059
368769438



In the example above, the input list has three CIDs, but the PUG-REST request returned five SIDs.  It means that some CID(s) must be associated with multiple SIDs, but it is hard to see which CID it is.  Therefore, we want the SIDs grouped by the corrsponding CIDs.  This can be done using the optional parameter "__list_return=grouped__" and changing the output format to __json__.

In [9]:
pugin   = "compound/cid/129825914,129742624,129783988"
pugoper = "sids"
pugout  = "json"
pugopt  = "list_return=grouped"
url     = prolog + '/' + pugin + '/' + pugoper + '/' + pugout + "?" + pugopt

res = requests.get(url)
json_output = res.text.strip().splitlines()
#print(res.text)
my_fun.scrollable_box(json_output, height='150px')

Note that the __json__ output format is used in the above request.  The "__txt__" output format in PUG-REST returns data into a single column but the result from the above request cannot fit well into a single column.

If you want output records to be "flattened", rather than being grouped by the input identifiers, use "**list_return=flat**".

In [10]:
pugopt  = "list_return=flat"
url     = prolog + '/' + pugin + '/' + pugoper + '/' + pugout + "?" + pugopt

res = requests.get(url)
print(res.text)

{
  "IdentifierList": {
    "SID": [
      341492923,
      341577059,
      341669951,
      368769438
    ]
  }
}



The default value for the "list_return" parameter is: 
- "flat" when the output format is TXT 
- "grouped" when the output format is JSON and XML

It is also possible to specify the input list **implicitly**, rather than providing the input identifiers explicitly.  For example, the following example uses a chemical name to specify the input list.

In [11]:
# Input CIDs are provided using a chemical name
url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/lactose/cids/txt'
res = requests.get(url)
cids = res.text.split()
print("# CIDs returned:", len(cids))
print(",".join(cids))

# Input CIDs are provided using the name, then coverted to SIDs.
url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/lactose/sids/txt'
res = requests.get(url)
sids = res.text.split()
print("# SIDs returned (method 1):", len(sids))
#print(",".join(sids))

# Input *SIDs* are provided using the name, and returned the input SIDs.
url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/substance/name/lactose/sids/txt'
res = requests.get(url)
sids = res.text.split()
print("# SIDs returned (method 2):", len(sids))
#print(",".join(sids))

# CIDs returned: 1
6134
# SIDs returned (method 1): 160
# SIDs returned (method 2): 163


The above example illustrates how the list conversion works.  
- In the first request, the name "lactose" is searched for against the Compound database and the resulting 7 CIDs are returned.
- If you change the operation part from "cids" to "sids" (as in the second request), the same name search is done first against the __Compound__ database, followed by the list conversion from the resulting 7 CIDs to associted 415 SIDs.
- In the third request, the name search is performed against the __Substance__ database, and the resulting 125 SIDs are returned.


**Exercise 1a** Statins are a class of drugs that lower cholesterol levels in the blood.  Retrieve in **JSON** the substance records associated with the compounds whose names contain the string "statin". 

- Make only one PUG-REST request.
- For partial name matching, set the *name_type* parameter to "word" (See the PUG-REST document for an example). 
- Group the substances by the corresponding compound records.
- Print the json output using print()

In [12]:
# Write your code in this cell.




## 2. Getting mixture/component molecules for a given molecule.

The list interconversion may be used to retrieve mixtures that contain a given molecule as a component.  To do this, the input molecule should be a single-component compound (that is, with only one covalently-bound unit), and the optional parameter "**cids_type=component**" should be provided.

In [19]:
prolog    = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"

url = prolog + "/compound/name/tylenol/cids/txt?cids_type=component"
res = requests.get(url)
cids = res.text.split()
print(len(cids))
#print( cids )

my_fun.scrollable_box(cids, height='150px')

510


<br>It should be noted that, if the input molecule is a multi-component compound, the option "**cids_type=component**" returns the components of that compound.  For example, the following example shows how to get all components of the first molecule in the "cids" list generated in the previous example. 

In [20]:
url = prolog + "/compound/cid/" + cids[0] + "/cids/txt?cids_type=component"
res = requests.get(url)
component_cids = res.text.split()
print( "CID:", cids[0])
print( "Number of Components", len(component_cids))
print( component_cids )

CID: 171814455
Number of Components 2
['134359418', '1983']


**Exercise 2a:** Many over-the-counter drugs contain more than one active ingredients.  In this exercise, we want to find component molecules that occur with three common pain killers (aspirin, tylenol, advil) as a mixture.

__Step 1.__ Define a list that contains three drug names (aspirin, tylenol, advil).

In [21]:
# Write your code in this cell.




__Step 2.__ Using a for loop, retrieve PubChem CIDs corresponding to the three drugs and store them in a new list.  In order not to overload the PubChem servers, stop the program for 0.2 second for each iteration in the for loop (using sleep()).

In [22]:
# Write your code in this cell.




__Step 3.__ Using another for loop, do the following things for each drug:
- Get the PubChem CIDs of the mixture compounds that contain each drug and store them in a list.
- Get the PubChem CIDs of the components that occur in any of the returned mixtures, by setting the "list_return" parameter to "flat".  This can be done with a single request.  
- Print all the components.
- Stop the code for 0.2 second using sleep() each time a PUG-REST request is made.

In [23]:
# Write your code in this cell.




## 3. Getting compounds tested in a given assay

PUG-REST may be used to retrieve compounds tested in a given assay.  For example, the following code cell shows how to get all compounds tested in AID 1207599.

In [25]:
url = prolog + "/assay/aid/" + "1207599" + "/cids/txt"
res = requests.get(url)
cids = res.text.split()
print(len(cids))
#print(cids)
my_fun.scrollable_box(cids, height='150px')

791


If you are interested in only the compounds that are tested "active" in a given assay, set the "**cids_type**" parameter to "**active**", as shwon in the code below.

In [27]:
url = prolog + "/assay/aid/" + "1207599" + "/cids/txt?cids_type=active"
res = requests.get(url)
cids = res.text.split()
print(len(cids))
#print(cids)
my_fun.scrollable_box(cids, height='150px')

393


It is also possible to specify the input assay list implicitly.  For example, the following code cell retrieves compounds tested in any assays targeting human Carbonic anhydrase 2 (CA2), whose accession number is P00918.

In [28]:
url = prolog + "/assay/target/accession/" + "P00918" + "/cids/txt"
res = requests.get(url)
cids = res.text.split()
print(len(cids))
#print(cids)
my_fun.scrollable_box(cids, height='150px')

30613


**Exercise 3a:** Find compounds that are tested to be active against human acetylcholinesterase (accession: P08173) and retrieve SMILES strings for those compounds.<br>  
- Split the CID list into smaller chunks (with a chunk size of 100).
- Print the retrieved data in a CSV format (CID and SMILES strings in the first and second columns, respectively).

In [ ]:
# Write your code in this cell.


